# EIA Demand data via API

In [1]:
import pandas as pd
import requests
import datetime
import json

from config import api_key

### Extract

In [2]:
# Use API to get demand dataset
url = f'https://api.eia.gov/series/?api_key={api_key}&series_id=EBA.CISO-ALL.D.HL'
response = requests.get(url)

In [3]:
# Review json for format
response_json = response.json()
# print(response_json)
num_resp=len(response_json['series'][0]['data'])
print(f'{num_resp} responses')
print(response_json['series'][0]['data'][0])
print(response_json['series'][0]['data'][-1])

39179 responses
['20191221T11-08', 27201]
['20150701T01-07', 31486]


In [4]:
# Pull data for dataframe from nested dictionary
dict = []
for response in range(num_resp):
    dict.append(response_json['series'][0]['data'][response])

In [5]:
# Create datafame and check results
df = pd.DataFrame(dict)
print(df.head())
print(df.tail())

                0        1
0  20191221T11-08  27201.0
1  20191221T10-08  23727.0
2  20191221T09-08  27201.0
3  20191221T08-08  27201.0
4  20191221T07-08  22539.0
                    0        1
39174  20150701T05-07  25661.0
39175  20150701T04-07  26388.0
39176  20150701T03-07  27416.0
39177  20150701T02-07  28989.0
39178  20150701T01-07  31486.0


### Transform

In [6]:
# Rename columns
demand_df = df.rename(columns={0:'timestamp', 1:'DEMAND'})
# Clean up timestamp by removing end string (08,07)
new = demand_df['timestamp'].str.split("-", n = 1, expand = True) 
demand_df['timestamp']= new[0] 
demand_df['timestamp'] = pd.to_datetime(demand_df['timestamp'])

# Splint datetimes as needed:
# demand_df['Year'] = demand_df['Timestamp'].dt.year
# demand_df['Month'] = demand_df['Timestamp'].dt.month
# demand_df['Day'] = demand_df['Timestamp'].dt.day
demand_df['Hour'] = demand_df['timestamp'].dt.hour # Hour 0 = midnight
demand_df['date'] = demand_df['timestamp'].dt.date

# There were duplicate listing for a handful of dates
# Dropped so not to violate primary key rules
demand_df.drop_duplicates(subset ="timestamp", 
                     keep = 'last', inplace = True) 

demand_df.head(24)

,timestamp,DEMAND,Hour,date
0,2019-12-21 11:00:00,27201.0,11,2019-12-21
1,2019-12-21 10:00:00,23727.0,10,2019-12-21
2,2019-12-21 09:00:00,27201.0,9,2019-12-21
3,2019-12-21 08:00:00,27201.0,8,2019-12-21
4,2019-12-21 07:00:00,22539.0,7,2019-12-21
5,2019-12-21 06:00:00,21366.0,6,2019-12-21
6,2019-12-21 05:00:00,20718.0,5,2019-12-21
7,2019-12-21 04:00:00,27201.0,4,2019-12-21
8,2019-12-21 03:00:00,20875.0,3,2019-12-21
9,2019-12-21 02:00:00,21439.0,2,2019-12-21


In [7]:
# In case this is helpful
demand_df.to_csv('eia_demand_data.csv')

### Load

In [8]:
from sqlalchemy import create_engine
from config import username, password

engine = create_engine('postgresql://{}:{}@localhost:5432/cali_renewable_db'.format(username,password))
con = engine.connect()

name_demand = "hourlydemand"
schema = 'Demand'
demand_df.to_sql(name=name_demand,con=con,schema=schema, if_exists='append',
                            index=False)